# Comapany Background:

Select  Collections,  Inc.,  was  a  start-up  subsidiary  of  a  major credit  card  company

***The company  purchased  distressed  consumer  debt  at  discounted  rates from  such  major  credit  card***
and  then  
used  data-driven  decision-making and dynamic value assessment to optimize the collection processes associated with the purchased accounts

For each purchased account, ***the first decision was whether to resell or attempt to collect***. For accounts the company decided to attempt to collect, a host of tactics was available for use in any sequence and with any frequency. 

Like other collection companies, Select Collections used ***the telephone  and  the  legal  system***  as  its  two  major  collection  tools.  



# Goal:


I want to see if we can ***build a model to predict how much money we will collect from  delinquent  accounts.***  Such  a  model  would  prove  very  useful in  ***deciding which accounts to purchase and how much to pay.***

There are ***3,570*** accounts in a data set that I will call the training set

***We  purchased  these  accounts  and  then  collected  the  amount  shown  in  the  last column, labeled totalpay.*** 
Totalpay is the variable of interest. I want you to ***come up with a way to forecast totalpay.***

I  want  you  to  use  the  model  you  come  up with to predict totalpay for the 3,570 accounts in a separate worksheet of data that I will call the test set. 

I  want  an  Excel  file  from  you  containing a single column of numbers. 
***In cell A1, please put the adjusted R-squared for your model.***
***In cells A2 through A3571, put your predicted values for the accounts in the test data set in ID order.***



# Datadict: 
![alt text](datadict.png "data dict")

# Data description:

![alt text](description.png "data description")

# Colume Comparation to total pay

![alt text](col_comparasion_totalpay.png "colume vs totalpay description")

# Col vs Col description
![alt text](col_to_col_relationship.png "col vs col description")

# Business Understanding


# Understanding the data
#### From data dict :
what col should be care?

#### from data description:
data distrbution is fair ? 
fair in total
rollout is also important ?
not fair

#### from col to total pay part we now only 4 cols are not well distributed

inacscr: The natural log of the ***accessscr***

***accessscr***: The account’s accessibility score, an a priori estimation of how likely it is that the accountholder is reached via phone.
bureauscr: The output of an in-house prediction model based on the likelihood of receiving payments from the accountholder using credit-bureau attributes as predictors 

numcalls: The number of telephone calls made (to date) to the accountholder

numrpcs: The number of “right-party connects” or phone calls in which the collection agent speaks with the accountholder 

#### how about col to col ?

what grid means to me ?


# Data Preparation

From ***Understanding the data parts***, already shown the data related accessibility is important

In [3]:

## import pacakge
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

# Load the data
data = pd.read_csv('Labeled_data.csv')

# Define features and target variable
X = data.drop(['acctid', 'totalpay'], axis=1)
y = data['totalpay']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)


# Modeling 

#### Representation

What algorithm we are using ? at least it support R-squared

Linear Regression: Linear regression is a simple and commonly used algorithm that predicts a continuous target variable based on one or more independent variables.

Polynomial Regression: Polynomial regression is a form of regression analysis in which the relationship between the independent variable x and the dependent variable y is modelled as an nth degree polynomial. 

In [ ]:
data = pd.read_csv('Labeled_data.csv')

# Define features and target variable
X = data[['state', 'zip','rollout','cobal','collscr','cs','accessscr','lnacscr','bureauscr','eaglemod','numcalls','numrpcs']]
y = data['totalpay']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)
X_train_encode = pd.get_dummies(X_train, columns=['state', 'zip', 'rollout', 'collscr'],drop_first=True)

## Train the data 
model = RandomForestRegressor(max_depth=6, min_samples_split=15, min_samples_leaf=5, random_state=1337)
model.fit(X_train_encode, y_train)



In [7]:
## use the test set 
X_test_encode = pd.get_dummies(X_test, columns=['state', 'zip', 'rollout', 'collscr'],drop_first=True)
X_test_encode = X_test_encode.reindex(columns=X_train_encode.columns, fill_value=0)
predictions = model.predict(X_test_encode)
X_test.to_excel('test_feature.xlsx', index=False)
y_test.to_excel('test_target.xlsx', index=False)
pd.DataFrame(predictions, columns=['totalpay']).to_excel('predict_target.xlsx', index=False)


#### Optimization

According to the stats it is weird. but the model is generate from professor. so no need to do Optimization 

#### Evaluation

Same as Optimization parts. but having some question on it.

***Q1 : What is the mean absolute percentage error (MAPE) of the predictions made by the deployed model? Enter your answer as a rounded integer. For example, if the MAPE is 20.6%, you should enter 21 (not 0.206).***

In [8]:
# Calculate mean squared error
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error on test set: {mse}")

# Calculate mean squared error
msep = mean_absolute_percentage_error(y_test, predictions)
print(f"Mean Absolute Percentage Error on test set: {msep*100} %")

Mean Squared Error on test set: 856513.9636756183
Mean Absolute Percentage Error on test set: 358.8362737193568 %


***Q2 : If we had perfect predictions for totalpay, what decision rule (expressed as an inequality) should we use to decide which accounts to purchase? If the inequality is met, we purchase; otherwise, we do not. For example, you might choose totalpay > 800.*** 


***Answer***: Since the reveunue = total_pay - cobal > 0  == revenue without operation cost 
and the operation cost is not visable on the case.  
So just simply take condition ***total_pay - cobal*0.25 > 0***

***Q3 Apply the decision rule you proposed in the previous question to the test set using the predictions from the deployed model.***

***Q3.1 What percentage of cases indicated a purchase, but actually resulted in a loss? (Enter your answer as a rounded integer (e.g., 82 for 81.64%).)***

In [9]:
#### ((prediction total_pay) - real total_pay - cobal < 0 
# Calculate the difference between the two arrays
condition = [pred - cobal*0.25 for pred, cobal in zip(predictions, X_test['cobal'])]
# Create a new DataFrame with the differences
condition_df = pd.DataFrame({'Action': condition, 'Diff': predictions - y_test })
# Print or display the DataFrame with differences

condition_df.to_excel('confusion_matrix.xlsx', index=False)


In [10]:
# Percentage
loss_df_percentage = condition_df[(condition_df['Action'] > 0) & (condition_df['Diff'] < 0)].shape[0] / condition_df.shape[0]
print(f"Loss Percentage Error on test set: {loss_df_percentage*100} %")

Loss Percentage Error on test set: 24.369747899159663 %


***Q3.2 What percentage of cases indicated not to purchase, but actually resulted in missed opportunities? (Enter your answer as a rounded integer (e.g., 82 for 81.64%).)***

In [11]:
##  total_pay - (prediction total_pay) > 0 == missed opportunites 
# Calculate the difference between the two arrays

# Create a new DataFrame with the differences
missed_opportunites_df_percentage = condition_df[(condition_df['Action'] < 0) & (condition_df['Diff'] > 0)].shape[0] / condition_df.shape[0]
print(f"Missed opportunites Percentage on test set: {missed_opportunites_df_percentage*100} %")

Missed opportunites Percentage on test set: 18.067226890756302 %


***Q4. Suppose we want to use this predictive model and decision rule as our data-driven solution to decide which of 3,570 accounts to buy. Based on the test set where you applied this solution, provide a 95% confidence interval for the expected profit. Round your final answer to the nearest 10,000 and express it as integers. For example, if the confidence interval goes from $-32,124.34 to $57,241.32, you would enter -30000 and 60000.***

# Business Perspective Question

***Q5 From a business perspective, do you believe this data-driven solution adds value? Please justify your answer.***

***Q6 In what follows, you'll work on improving the current solution for selecting accounts to purchase. Choose an appropriate performance measure (e.g., accuracy, precision, recall) to assess your progress. Briefly justify your choice.***